In [44]:
##Importing the data set

In [2]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t',
header=None, names=['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

In [23]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1, random_state=42)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [24]:
training_set['Label'].value_counts(normalize=True)

ham     0.866981
spam    0.133019
Name: Label, dtype: float64

In [25]:
test_set['Label'].value_counts(normalize=True)

ham     0.861759
spam    0.138241
Name: Label, dtype: float64

In [26]:
# Before cleaning
training_set.head(3)

,Label,SMS
0,ham,Squeeeeeze!! This is christmas hug.. If u lik ...
1,ham,And also I've sorta blown him off a couple tim...
2,ham,Mmm thats better now i got a roast down me! i...


### removig punctuations and converting to lower case

In [27]:
# After cleaning
training_set['SMS'] = training_set['SMS'].str.replace(
   '\W', ' ') # Removes punctuation
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head(4)


/tmp/ipykernel_12522/2819761578.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str.replace(


,Label,SMS
0,ham,squeeeeeze this is christmas hug if u lik ...
1,ham,and also i ve sorta blown him off a couple tim...
2,ham,mmm thats better now i got a roast down me i ...


In [28]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
   for word in sms:
      vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [29]:
len(vocabulary)

7816

In [31]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
   for word in sms:
      word_counts_per_sms[word][index] += 1

In [32]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,cappuccino,msgrcvd,al,while,ask,unfortunately,dontcha,jobyet,countin,08450542832,...,grownup,middle,hai,witout,1843,07973788240,xxxmobilemovieclub,uh,pie,ahhh
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,cappuccino,msgrcvd,al,while,ask,unfortunately,dontcha,jobyet,...,grownup,middle,hai,witout,1843,07973788240,xxxmobilemovieclub,uh,pie,ahhh
0,ham,"[squeeeeeze, this, is, christmas, hug, if, u, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[and, also, i, ve, sorta, blown, him, off, a, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[mmm, thats, better, now, i, got, a, roast, do...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[mm, have, some, kanji, dont, eat, anything, h...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[so, there, s, a, ring, that, comes, with, the...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

In [35]:
# Initiate parameters of number of unique words in spam message 
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
   n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
   p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
   parameters_spam[word] = p_word_given_spam

   n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
   parameters_ham[word] = p_word_given_ham
   

In [36]:
import re

def classify(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham: 
         p_ham_given_message *= parameters_ham[word]

   print('P(Spam|message):', p_spam_given_message)
   print('P(Ham|message):', p_ham_given_message)

   if p_ham_given_message > p_spam_given_message:
      print('Label: Ham')
   elif p_ham_given_message < p_spam_given_message:
      print('Label: Spam')

      

In [37]:
classify("Sounds good, Tom, then see u there")


P(Spam|message): 4.74693224259436e-25
P(Ham|message): 2.7878169823581606e-21
Label: Ham


In [38]:
def classify_test_set(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham:
         p_ham_given_message *= parameters_ham[word]

   if p_ham_given_message > p_spam_given_message:
      return 'ham'
   elif p_spam_given_message > p_ham_given_message:
      return 'spam'
  

In [39]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()


,Label,SMS,predicted
0,ham,Was playng 9 doors game and gt racing on phone...,ham
1,ham,I dont thnk its a wrong calling between us,ham
2,ham,All e best 4 ur exam later.,ham
3,ham,Hey what how about your project. Started aha da.,ham
4,ham,"Dunno, my dad said he coming home 2 bring us o...",ham


In [43]:

correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
   row = row[1]
   if row['Label'] == row['predicted']:
      correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Tp=0
Tn=0
Fp=0
Fn=0
total=test_set.shape[0]

for row in test_set.iterrows():
    row=row[1]
    if row['Label'] == row['predicted']:
       if(row['Label']== 'ham'):
          Tp+=1
       else:
          Tn+=1
    else:
      if row['Label'] == 'ham':
         Fn+=1
      elif row['Label'] == 'spam':
         Fp+=1

print("Tp=",Tp," ","FP",Fp,"",)
print("Fn=",Fn," ","Tn",Tn,"",)
precison= Tp/(Tp+Fp)
recall=Tp/(Tp+Fn)
f1_score=2*precison*recall/(precison+recall)
print("Precision",precison)
print("recall",recall)
print("f1score",f1_score)

Correct: 1092
Incorrect: 22
Accuracy: 0.9802513464991023
Tp= 951   FP 13 
Fn= 9   Tn 141 
Precision 0.9865145228215768
recall 0.990625
f1score 0.9885654885654885
